# Data processing and empirical modelling

What needs to be presented:
- Read CSV (local and remote)
- @select
- @filter
- @mutate, @transmute
- @rename
- @left_join, @inner_join, @outer_join
- @pivot_wider, @pivot_longer
- @group_by and @summarise

In [17]:
using Tidier
import DataFrames

In [ ]:
#### Snippets to consider

# using DBnomics
# natl_acc = rdb("Eurostat", "nama_10_gdp", mask="B1GQ")

# using HTTP
# res = HTTP.get("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true")

# import Downloads
# using CSV
# natl_acc = CSV.read(Downloads.download("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true"), DataFrame)

In [31]:
using CSV
natl_acc = CSV.read("datasets/estat_nama_10_gdp.tsv.gz", DataFrame; delim="\t")

Row,"freq,unit,na_item,geo\\TIME_PERIOD",1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
,String,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15
1,"A,CLV05_MEUR,B1G,AT",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,177617.0,180999.4,184799.1,192025.4,198247.2,205898.4,208771.6,212101.1,214463.0,220728.8,225888.1,234396.7,243454.7,247429.8,237060.0,241601.7,249489.2,250808.6,251167.1,252879.7,255017.1,259996.4,266016.9,272985.2,276798.3,258800.9,267789.4,281637.3,278959.7
2,"A,CLV05_MEUR,B1G,BA",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,6112.0,6263.5,6583.6,6848.4,7305.1,7613.8,8065.0,8559.9,9042.7,8817.8,8882.4,8974.7,8903.7,9118.0,9224.4,9358.7 b,9658.1,9978.2,10355.7,10650.6,10358.6,11080.5,11541.8,:
3,"A,CLV05_MEUR,B1G,BE",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,220134.8,222151.7,230253.3,234553.5,242356.1,251047.3,254864.1,259025.9,261911.7,270219.4,276828.7,283611.6,294188.4,296506.7,290513.2,298892.4,304824.0,307227.5,308495.4,313684.3,320657.4,323677.5,328736.7,334655.5,342451.4,325917.8,346988.5,357646.6 p,362628.5 p
4,"A,CLV05_MEUR,B1G,BG",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,18104.4,19181.0,16279.8,16583.7,15103.9,15700.3,16339.7,17581.4,18311.9,19205.0,20444.8,21905.4,23339.9,24775.6,24562.9,25012.4,25487.8,25552.2,25131.1,25372.8,26119.5,26792.6,27671.7,28637.7,29703.4,28514.7,30803.2,32447.5,32696.8 p
5,"A,CLV05_MEUR,B1G,CH",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,270973.2,269132.6,269249.4,268649.6,271547.9,272766.6,273767.0,279174.0,287525.2,290918.8,301587.8,306917.1,307431.2,307261.6,315229.8,323882.1,336810.8,349722.2,359861.1,351613.1,362699.3,369754.6,374698.2,381505.9,390405.6,396747.1,405030.0,410565.5,422877.6,427969.5,419196.5,441816.0,453061.4,456738.6
6,"A,CLV05_MEUR,B1G,CY",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,8788.7,8898.4,9133.4,9691.5,10175.9,10782.9,11209.1,11626.5,11931.4,12531.1,13139.3,13758.7,14460.0,14987.3,14685.3,15026.0,15088.7,14568.5,13608.8,13367.1,13802.8,14710.2,15556.1,16438.0,17371.7,16782.6,18446.0,19379.0 p,19856.3 p
7,"A,CLV05_MEUR,B1G,CZ",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,76060.0,79047.8,78073.5,77812.0,78923.7,82298.7,84825.4,86543.6,89117.8,93443.9,99776.8,107041.5,112700.6,116727.7,110565.0,113884.7,115884.8,114912.8,114894.5,118178.4,123876.2,126971.0,133574.6,138128.1,142281.3,134791.0,139402.8,142668.2,143114.1
8,"A,CLV05_MEUR,B1G,DE",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,1720691.5,1753676.0,1735271.6,1771602.4,1803630.8,1821557.2,1858605.0,1898999.1,1931266.5,1993889.2,2034044.3,2035239.4,2022571.4,2055555.9,2069658.0,2149968.1,2226214.9,2249638.6,2109813.0,2201835.0,2286925.5,2299115.4,2309632.2,2361738.2,2390181.3,2443960.4,2512319.6,2535982.4,2557494.0 p,2453999.1 p,2535026.3 p,2578049.6 p,2576137.4 p
9,"A,CLV05_MEUR,B1G,DK",92152.9,97112.5,99054.8,101270.9,105455.4,105414.2,105277.0,109326.1,111899.2,116666.3,121055.9,125339.7,126182.8,127910.5,129527.0,132120.4,133661.5,135969.4,136654.2,142962.7,147450.6,151214.3,155933.0,158960.7,164179.7,171867.0,173312.8,173854.2,174517.8,177968.6,180849.0,187788.5,188728.5,188831.4,180433.2,183751.9,186593.6,187000.9,188719.7,191638.4,195803.8,202021.6,208023.8,211654.5,214785.0,208686.7,223115.7,231100.7,235996.6


In [35]:
natl_acc = DataFrames.rename(natl_acc, "freq,unit,na_item,geo\\TIME_PERIOD" => "mixed")

Row,mixed,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
,String,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15
1,"A,CLV05_MEUR,B1G,AT",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,177617.0,180999.4,184799.1,192025.4,198247.2,205898.4,208771.6,212101.1,214463.0,220728.8,225888.1,234396.7,243454.7,247429.8,237060.0,241601.7,249489.2,250808.6,251167.1,252879.7,255017.1,259996.4,266016.9,272985.2,276798.3,258800.9,267789.4,281637.3,278959.7
2,"A,CLV05_MEUR,B1G,BA",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,6112.0,6263.5,6583.6,6848.4,7305.1,7613.8,8065.0,8559.9,9042.7,8817.8,8882.4,8974.7,8903.7,9118.0,9224.4,9358.7 b,9658.1,9978.2,10355.7,10650.6,10358.6,11080.5,11541.8,:
3,"A,CLV05_MEUR,B1G,BE",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,220134.8,222151.7,230253.3,234553.5,242356.1,251047.3,254864.1,259025.9,261911.7,270219.4,276828.7,283611.6,294188.4,296506.7,290513.2,298892.4,304824.0,307227.5,308495.4,313684.3,320657.4,323677.5,328736.7,334655.5,342451.4,325917.8,346988.5,357646.6 p,362628.5 p
4,"A,CLV05_MEUR,B1G,BG",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,18104.4,19181.0,16279.8,16583.7,15103.9,15700.3,16339.7,17581.4,18311.9,19205.0,20444.8,21905.4,23339.9,24775.6,24562.9,25012.4,25487.8,25552.2,25131.1,25372.8,26119.5,26792.6,27671.7,28637.7,29703.4,28514.7,30803.2,32447.5,32696.8 p
5,"A,CLV05_MEUR,B1G,CH",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,270973.2,269132.6,269249.4,268649.6,271547.9,272766.6,273767.0,279174.0,287525.2,290918.8,301587.8,306917.1,307431.2,307261.6,315229.8,323882.1,336810.8,349722.2,359861.1,351613.1,362699.3,369754.6,374698.2,381505.9,390405.6,396747.1,405030.0,410565.5,422877.6,427969.5,419196.5,441816.0,453061.4,456738.6
6,"A,CLV05_MEUR,B1G,CY",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,8788.7,8898.4,9133.4,9691.5,10175.9,10782.9,11209.1,11626.5,11931.4,12531.1,13139.3,13758.7,14460.0,14987.3,14685.3,15026.0,15088.7,14568.5,13608.8,13367.1,13802.8,14710.2,15556.1,16438.0,17371.7,16782.6,18446.0,19379.0 p,19856.3 p
7,"A,CLV05_MEUR,B1G,CZ",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,76060.0,79047.8,78073.5,77812.0,78923.7,82298.7,84825.4,86543.6,89117.8,93443.9,99776.8,107041.5,112700.6,116727.7,110565.0,113884.7,115884.8,114912.8,114894.5,118178.4,123876.2,126971.0,133574.6,138128.1,142281.3,134791.0,139402.8,142668.2,143114.1
8,"A,CLV05_MEUR,B1G,DE",:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,:,1720691.5,1753676.0,1735271.6,1771602.4,1803630.8,1821557.2,1858605.0,1898999.1,1931266.5,1993889.2,2034044.3,2035239.4,2022571.4,2055555.9,2069658.0,2149968.1,2226214.9,2249638.6,2109813.0,2201835.0,2286925.5,2299115.4,2309632.2,2361738.2,2390181.3,2443960.4,2512319.6,2535982.4,2557494.0 p,2453999.1 p,2535026.3 p,2578049.6 p,2576137.4 p
9,"A,CLV05_MEUR,B1G,DK",92152.9,97112.5,99054.8,101270.9,105455.4,105414.2,105277.0,109326.1,111899.2,116666.3,121055.9,125339.7,126182.8,127910.5,129527.0,132120.4,133661.5,135969.4,136654.2,142962.7,147450.6,151214.3,155933.0,158960.7,164179.7,171867.0,173312.8,173854.2,174517.8,177968.6,180849.0,187788.5,188728.5,188831.4,180433.2,183751.9,186593.6,187000.9,188719.7,191638.4,195803.8,202021.6,208023.8,211654.5,214785.0,208686.7,223115.7,231100.7,235996.6


In [44]:
nat_acc = @chain natl_acc begin
    @separate(mixed, (freq,unit,na_item,geo), ",")
    @pivot_longer(-(freq:geo), names_to=year,values_to=value)
    @select(-freq)
    @mutate(value = as_float(value), year = as_integer(str_squish(year)))
end

Row,unit,na_item,geo,year,value
,SubStrin…,SubStrin…,SubStrin…,Int64,Float64?
1,CLV05_MEUR,B1G,AT,1975,missing
2,CLV05_MEUR,B1G,BA,1975,missing
3,CLV05_MEUR,B1G,BE,1975,missing
4,CLV05_MEUR,B1G,BG,1975,missing
5,CLV05_MEUR,B1G,CH,1975,missing
6,CLV05_MEUR,B1G,CY,1975,missing
7,CLV05_MEUR,B1G,CZ,1975,missing
8,CLV05_MEUR,B1G,DE,1975,missing
9,CLV05_MEUR,B1G,DK,1975,92152.9


In [50]:
bg_nat_acc = @chain nat_acc begin
    @filter(geo == "BG", unit ==  "CLV10_MNAC", na_item in ("B1GQ", "P51G"))
    @select(na_item,year,value)
    @drop_missing()
end

Row,na_item,year,value
,SubStrin…,Int64,Float64
1,B1GQ,1995,53253.3
2,P51G,1995,4874.4
3,B1GQ,1996,56025.4
4,P51G,1996,1575.2
5,B1GQ,1997,48117.2
6,P51G,1997,3945.4
7,B1GQ,1998,49940.4
8,P51G,1998,5232.7
9,B1GQ,1999,45747.3


In [ ]:
# names(nat_acc)
println(@distinct(nat_acc.na_item))

SubString{String}["B1G", "B1GQ", "D21", "D21X31", "D31", "P3", "P31_S13", "P31_S14", "P31_S14_S15", "P31_S15", "P32_S13", "P3_P5", "P3_P6", "P3_S13", "P41", "P51G", "P5G", "P6", "P61", "P62", "P7", "P71", "P72", "P52_P53", "B11", "B111", "B112", "B2A3G", "D1", "D11", "D12", "D2", "D2X3", "D3", "P52", "P53", "YA0", "YA1", "YA2"]
